## Mount G-Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('/content/gdrive/My Drive/flt/Multivariate/')

## Import Libraries and Files

In [3]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.15.0


Using TensorFlow backend.


In [4]:
df = pd.read_csv('consumer_complaints.csv', encoding='latin-1')
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [5]:
#df['Consumer complaint narrative'] = df.consumer_complaint_narrative
#df['Product'] = df.product
col = ['consumer_complaint_narrative', 'product']
df = df[col]
df = df[pd.notnull(df['consumer_complaint_narrative'])]
df.head()

,consumer_complaint_narrative,product
190126,XXXX has claimed I owe them {$27.00} for XXXX ...,Debt collection
190135,Due to inconsistencies in the amount owed that...,Consumer Loan
190155,In XX/XX/XXXX my wages that I earned at my job...,Mortgage
190207,I have an open and current mortgage with Chase...,Mortgage
190208,XXXX was submitted XX/XX/XXXX. At the time I s...,Mortgage


In [6]:
df.isnull().sum()

consumer_complaint_narrative    0
product                         0
dtype: int64

In [7]:
df['product'].value_counts()

Debt collection            17552
Mortgage                   14919
Credit reporting           12526
Credit card                 7929
Bank account or service     5711
Consumer Loan               3678
Student loan                2128
Prepaid card                 861
Payday loan                  726
Money transfers              666
Other financial service      110
Name: product, dtype: int64

## Train test split

In [8]:
# Split data into train and test
train_size = int(len(df) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(df) - train_size))

Train size: 53444
Test size: 13362


In [0]:
train_narrative = df['consumer_complaint_narrative'][:train_size]
train_product = df['product'][:train_size]

test_narrative = df['consumer_complaint_narrative'][train_size:]
test_product = df['product'][train_size:]

In [0]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [0]:
tokenize.fit_on_texts(train_narrative) # only fit on train
x_train = tokenize.texts_to_matrix(train_narrative)
x_test = tokenize.texts_to_matrix(test_narrative)

In [0]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_product)
y_train = encoder.transform(train_product)
y_test = encoder.transform(test_product)

In [0]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [18]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (53444, 1000)
x_test shape: (13362, 1000)
y_train shape: (53444, 11)
y_test shape: (13362, 11)


## Model Building

In [0]:
batch_size = 32
epochs = 5

In [20]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [21]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 48099 samples, validate on 5345 samples
Epoch 1/5





48099/48099 [==============================] - 14s 298us/step - loss: 0.7065 - acc: 0.7846 - val_loss: 0.5769 - val_acc: 0.8185
Epoch 2/5
48099/48099 [==============================] - 13s 280us/step - loss: 0.5024 - acc: 0.8402 - val_loss: 0.5539 - val_acc: 0.8290
Epoch 3/5
48099/48099 [==============================] - 13s 276us/step - loss: 0.4294 - acc: 0.8615 - val_loss: 0.5475 - val_acc: 0.8324
Epoch 4/5
48099/48099 [==============================] - 13s 280us/step - loss: 0.3645 - acc: 0.8826 - val_loss: 0.5518 - val_acc: 0.8303
Epoch 5/5
48099/48099 [==============================] - 13s 279us/step - loss: 0.3031 - acc: 0.9020 - val_loss: 0.5751 - val_acc: 0.8262


## Model Validation

In [22]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

13362/13362 [==============================] - 1s 52us/step
Test score: 0.647031471751047
Test accuracy: 0.8033976949290804


## Actual vs Prediction

In [23]:
# Here's how to generate a prediction on individual examples
text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_narrative.iloc[i][:50], "...")
    print('Actual label:' + test_product.iloc[i])
    print("Predicted label: " + predicted_label + "\n")

XXXX. I had intended to purchase a home warranty t ...
Actual label:Debt collection
Predicted label: Debt collection

I received a call from Receivable Management Servi ...
Actual label:Debt collection
Predicted label: Debt collection

The amount of {$51.00} dollars was added to my cre ...
Actual label:Debt collection
Predicted label: Credit reporting

On XX/XX/XXXX, I decided to close my Capital One c ...
Actual label:Credit card
Predicted label: Credit card

I disputing the information XXXX XXXX placed on my ...
Actual label:Credit reporting
Predicted label: Credit reporting

In XXXX of 2015, Discover Bank 's customer service ...
Actual label:Bank account or service
Predicted label: Bank account or service

The incorrect address of XXXX XXXX XXXX, XXXX XXXX ...
Actual label:Credit reporting
Predicted label: Credit reporting

I am being contacted by XXXX XXXX XXXX about an ac ...
Actual label:Credit card
Predicted label: Debt collection

Did n't get score 
 ...
Actual label:Credit rep